## `POST /train_model`
**URL**  

http://192.5.87.49:9090/train_model


**Description**  
Submits a Ray job to train one of your five models. Under the hood it invokes the corresponding `train*.py` script via `ray job submit`.

**Query Parameters**  
- `model_name` (required)  
  – One of: `resnet50`, `rawcnn`, `emb_mlp`, `effb3_lora`, `meta_mlp`  
- `batch_size` (optional, default `64`)  
- `lr` (optional, default `1e-4`)  
- `weight_decay` (optional, default `1e-4`)  
- `epochs` (optional, default `20`)  
- `num_gpus` (optional) – override default GPU fraction  
- `num_cpus` (optional) – override default CPU count  

**Example**  
```bash
curl -X POST "http://192.5.87.49:9090/train_model?model_name=resnet50&batch_size=32&lr=0.0001&epochs=10"
```


## `POST /retrain_model`
**URL**  

http://192.5.87.49:9090/retrain_model


**Description**  
Same as `/train_model`, but runs a “retrain” script to fine‑tune existing heads fetched from the MLflow registry.

**Query Parameters**  
- `model_name` (required)  
  – One of: `resnet50`, `rawcnn`, `emb_mlp`, `effb3_lora`, `meta_mlp` 
- `epochs` (optional, default `20`)  
- `num_gpus` (optional) – override default GPU fraction  
- `num_cpus` (optional) – override default CPU count
- `emb_model_name`, `raw_model_name`, `res_model_name`, `eff_model_name`, `meta_model_name` (all optional, defaults point to your registered MLflow model names)

**Example**  
```bash
curl -X POST "http://192.5.87.49:9090/retrain_model?model_name=meta_mlp&epochs=5"
```


## `POST /tune_effb3`
**URL**  

http://192.5.87.49:9090/tune_effb3


**Description**  
Launches a Ray Tune run (using ASHA) to search hyperparameters for EfficientNet‑B3‑LoRA.

**Query Parameters** 
- `epochs` (optional, default `20`)  
- `num_samples` (optional, default `20`) – Number of trials
- `num_gpus` (optional) – default 1.0 
- `num_cpus` (optional) – default 4

**Example**  
```bash
curl -X POST "http://192.5.87.49:9090/tune_effb3?epochs=30&num_samples=50"
```


## `POST /tune_emb_mlp`
**URL**  

http://192.5.87.49:9090/tune_emb_mlp


**Description**  
Same as `/tune_effb3` but targets your PANNs‑Embedding MLP with Ray Tune + ASHA.

**Query Parameters** 
- `epochs` (optional, default `30`)  
- `num_samples` (optional, default `20`) – Number of trials
- `num_gpus` (optional) – default 1.0 
- `num_cpus` (optional) – default 4

**Example**  
```bash
curl -X POST "http://192.5.87.49:9090/tune_emb_mlp?num_samples=10"
```


## `GET /status`
**URL**  

http://192.5.87.49:9090/status


**Description**  
Queries Ray for a job’s current status. If it’s `SUCCEEDED`, it will also scrape the logs for an `MLFLOW_RUN_ID`.

**Query Parameters** 
- `job_id` (required) – the Ray job UUID returned by any of the above submit endpoints

**Example**  
```bash
curl -X POST "http://192.5.87.49:9090/status?job_id=1234-5678-abcd"
```


## `POST /train`
**URL**  

http://192.5.87.49:9090/train


**Description**  
Starts an MLflow run in the background (simulated long work) and immediately returns a tracking URL. Only one run can be active at a time—additional calls will return `{"status":"in_progress"}`.

**Query Parameters** 
- `model_name` (required) – arbitrary tag you want to log
- `data_source` (required) – tag for your data provenance

**Example**  
```bash
curl -X POST "http://192.5.87.49:9090/train?model_name=rawcnn&data_source=s3://my-bucket/train"
```


# [ADDITIONAL] - Keep Track of Experiment training status from bash

## Pool MLFLOW

``` bash
RUN_ID="2767d8907bdf4b48a22909b525677f76"

# terminal states to stop polling on
TERMINAL="FINISHED|FAILED|KILLED"

while true; do
  status=$(curl -s \
    "${MLFLOW_TRACKING_URI}/api/2.0/mlflow/runs/get?run_id=${RUN_ID}" \
    | jq -r '.run.info.status')

  echo "$(date +%T) → run ${RUN_ID} status: ${status}"

  if [[ "$status" =~ ^(${TERMINAL})$ ]]; then
    echo "Run reached terminal state: $status"
    break
  fi

  sleep 10
done
```

## Pool FastAPI- server with Ray JobID

``` bash
JOB_ID="$1"
STATUS_URL="http://192.5.87.49:9090/status?job_id=${JOB_ID}"

echo "Tracking Ray job ${JOB_ID} at ${STATUS_URL}"

while true; do
  # Fetch status JSON
  RESP=$(curl -s "$STATUS_URL")
  STATUS=$(echo "$RESP" | jq -r .status)

  printf "%s → job %s status: %s\n" "$(date +'%T')" "$JOB_ID" "$STATUS"

  if [ "$STATUS" = "SUCCEEDED" ]; then
    # Extract MLflow run ID if present
    MLFLOW_ID=$(echo "$RESP" | jq -r '.mlflow_run_id // empty')
    if [ -n "$MLFLOW_ID" ]; then
      echo "Job succeeded. MLflow run ID: $MLFLOW_ID"
    else
      echo "Job succeeded. (No MLflow run ID found in response.)"
    fi
    exit 0

  elif [[ "$STATUS" =~ ^(FAILED|STOPPED)$ ]]; then
    echo "Job ${JOB_ID} ended with status: $STATUS"
    exit 1
  fi

  sleep 10
done
```